In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import re

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
mp_holistic = mp.solutions.holistic

df = pd.DataFrame()

In [ ]:
# Regular expression pattern to extract coordinates
pattern = r'x: ([-+]?[0-9]*\.?[0-9]+)\ny: ([-+]?[0-9]*\.?[0-9]+)\nz: ([-+]?[0-9]*\.?[0-9]+)'

def get_coordinates(input_string):
    # Extract coordinates using regex
    matches = re.findall(pattern, input_string)

    # Convert the extracted coordinates to a list of dictionaries
    coordinates = [{'x': float(match[0]), 'y': float(match[1]), 'z': float(match[2])} for match in matches]
    return coordinates

In [ ]:
import os
path = os.getcwd()

In [ ]:
def extract_video(filename):
    global df
    video = f'{path}/{filename}.mp4'
    # For webcam input:
    cap = cv2.VideoCapture(video)
    results = None
    with mp_holistic.Holistic(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame. Or video finished. ")
                # If loading a video, use 'break' instead of 'continue'.
                break

            results = hands.process(image)

            # # Draw the hand annotations on the image.
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if (results.right_hand_landmarks):
                array = get_coordinates(str(results.right_hand_landmarks.landmark))
                new_row = {}
                for i in range(len(array)):
                    new_row[f'x_right_hand_{i}'] = array[i]["x"]
                    new_row[f'y_right_hand_{i}'] = array[i]["y"]

                
                df = df.append(new_row, ignore_index=True)

                # mp_drawing.draw_landmarks(
                #     image,
                #     results.right_hand_landmarks,
                #     mp_hands.HAND_CONNECTIONS,
                #     mp_drawing_styles.get_default_hand_landmarks_style(),
                #     mp_drawing_styles.get_default_hand_connections_style())
            
            if (results.left_hand_landmarks):
                array = get_coordinates(str(results.left_hand_landmarks.landmark))
                new_row = {}
                for i in range(len(array)):
                    new_row[f'x_left_hand_{i}'] = array[i]["x"]
                    new_row[f'y_left_hand_{i}'] = array[i]["y"]
                
                df = df.append(new_row, ignore_index=True)

                # mp_drawing.draw_landmarks(
                #     image,
                #     results.left_hand_landmarks,
                #     mp_hands.HAND_CONNECTIONS,
                #     mp_drawing_styles.get_default_hand_landmarks_style(),
                #     mp_drawing_styles.get_default_hand_connections_style())
                
                # Flip the image horizontally for a selfie-view display.
                # cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
                # if cv2.waitKey(5) & 0xFF == 27:
                #   break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
filename = "Hello"
extract_video(filename)
df.to_parquet(f'{filename}.parquet')

In [ ]:
pd.read_parquet(f'{filename}.parquet', engine='pyarrow')